# Library Import

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 100)
import requests
import re
import json
from tqdm import tqdm_notebook as tqdm
import time

# Ingest CSG / ICOMS data

In [2]:
df_csg = pd.read_csv('data/CSG_RAW.csv',low_memory=False)
df_icoms = pd.read_csv('data/ICOMS_RAW.csv',low_memory=False)

In [3]:
df_csg.head()

,BILLER,SYS,PRIN,AGENT,SUB_ACCT_NO,CUSTOMER_NAME,CUST_TYPE,VIP_FLAG,DWELL_TYPE,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIP_CODE,MDW Vlookup,KAC Vlookup,East Raw Data
0,CHR,8245.0,1100.0,10.0,8245110010025070,"THEISEN JR HIGH SCHOL,*",COM,4-CB PARTIAL COMP,C7-CB:EDUCTNK-12,525 E PIONEER RD,,FOND DU LAC,WI,54935-6451,NaN,NaN,NaN
1,CHR,8245.0,1100.0,10.0,8245110010046250,"RIVERSIDE ELEMENTARY SCH,*",COM,4-CB PARTIAL COMP,C7-CB:EDUCTNK-12,396 LINDEN ST,,FOND DU LAC,WI,54935-4925,NaN,NaN,NaN
2,CHR,8245.0,1100.0,10.0,8245110010050100,"PIER SCHOOL,*",COM,4-CB PARTIAL COMP,C7-CB:EDUCTNK-12,259 OLD PIONEER RD,,FOND DU LAC,WI,54935-6150,NaN,NaN,NaN
3,CHR,8245.0,1100.0,10.0,8245110010059600,"ROBERTS SCHOOL,*",COM,4-CB PARTIAL COMP,C7-CB:EDUCTNK-12,270 CANDY LN,,FOND DU LAC,WI,54935-5323,NaN,NaN,NaN
4,CHR,8245.0,1100.0,10.0,8245110010091950,"EVANS SCHOOL,*",COM,4-CB PARTIAL COMP,C7-CB:EDUCTNK-12,140 S PETERS AVE,,FOND DU LAC,WI,54935-3830,NaN,NaN,NaN


In [4]:
df_icoms.head()

,BILLER,SITE_ID,ACCOUNT_NUMBER,CUSTOMER_NAME,CUSTOMER_STATUS,STATUS_DATE,INSTALL_DATE,CUSTOMER_TYPE,CUSTOMER_CATEGORY,DWELLING_TYPE,BILL_TYPE,ASSOC_CODE,STREET_ADDRESS,BUILDING,CITY,STATE,ZIP_CODE,COMPANY,DIVISION,FRANCHISE,TOTAL_MRR,TOTAL_AR_BAL,AR_UNDER_30_DAYS,AR_31_TO_60_DAYS,AR_OVER_60_DAYS,LAST_PMT_DATE,MDW vlookup,KAC Vlookup,East Vlookup
0,OHI,2,906403,FIRE DEPT #16,ACTIVE,2010-09-21,1978-12-01,# - ENT FRACHISE,Y - VERTICAL GOV,G - GOVERNMENT,C,FRANPART,1130 E WEBER RD,NaN,COLUMBUS,OH,43211,15,57,606,70.0,75.26,75.26,0.0,0.0,2019-09-04,NaN,NaN,NaN
1,OHI,2,3388503,ALLEN COUNTY EDUCATION CENTER,ACTIVE,2000-10-02,2000-10-02,# - ENT FRACHISE,D - GOVERNMENT/PUBLIC,S - SCHOOL,S,FRANSCHL,136 S WEST ST,NaN,LIMA,OH,45801,5,28,401,0.0,0.00,0.00,0.0,0.0,NaN,True,NaN,NaN
2,OHI,2,3699901,NORTHSIDE MENNONITE SCHOOL,ACTIVE,1999-03-25,1979-10-31,# - ENT FRACHISE,D - GOVERNMENT/PUBLIC,S - SCHOOL,S,FRANSCHL,1318 N MAIN ST,NaN,LIMA,OH,45801,5,28,401,0.0,0.00,0.00,0.0,0.0,1997-09-26,True,NaN,NaN
3,OHI,2,5888701,NEIL ARMSTRONG MUSEUM,ACTIVE,1988-07-28,1988-07-28,# - ENT FRACHISE,D - GOVERNMENT/PUBLIC,G - GOVERNMENT,S,FRANAGRE,500 APOLLO DR,NaN,WAPAKONETA,OH,45895,5,28,811,0.0,0.00,0.00,0.0,0.0,NaN,True,NaN,NaN
4,OHI,2,6948501,FIREHOUSE,ACTIVE,1998-12-14,1998-12-14,# - ENT FRACHISE,G - RESIDENTIAL,G - GOVERNMENT,S,FIREHOUS,7698 OBERLIN CT NW,NaN,LANCASTER,OH,43130,15,19,630,0.0,0.00,0.00,0.0,0.0,NaN,True,NaN,NaN


# API Query Function
You will need to hardcode your API key from google (https://developers.google.com/places/web-service/get-api-key) into the hyperparameter 'API_KEY'

In [5]:
def single_query(address_line1, address_line2=None, city=None, state=None, zipCd=None, name=None, 
                 API_KEY=''):
    '''
    Function for intereacting with the Places /textsearch/ API with a single query
    
    Parameters
    ----------
    address_line1 (str): street address line 1 string
    address_line2 (str) : street address line 2 string (optional)
    city (str) : city str (optional)
    state (str) : state str (optional)
    zipCd (str or obj) : Zip Code (optional)
    name (str) : name of customer (optional)
    API_KEY : API Key. Required for using the Google Places API
    
    Returns
    -------
    dictionary object: dictionary with key data fields returned from the places API
                        {'google_name' : name, 
                        'google_formatted_address' : formatted_address, 
                        'google_establishment_types' : type, 
                        'google_permanently_closed' : permanently_closed,
                        'google_photo_reference_id' : photo_reference,
                        'google_place_id' : place_id,
                        'google_api_log' : a status record of how the data was retrieved,}
                        
    For more info, see: https://developers.google.com/places/web-service/place-data-fields 
    
    '''
    URL = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
    req_count = 0
    
    #clean name, dropping numeric and special characters:
    name = re.sub('[^a-zA-Z \n\.]', '', name).strip()
    
    #clean the address data. Convert all to strings and drop NaN values:
    query_list = [name, address_line1,address_line2,city,state,zipCd]
    query_list = [str(i) for i in query_list if not isinstance(i,type(None)) and str(i) !='nan']
    
    #string together the address, dropping Nones:
    full_query = ' '.join(query_list)
    
    #request findplacefromtext:
    payload = {'key' : API_KEY,
               'input' : full_query,
               'inputtype' : 'textquery',
               'fields' : 'photos,formatted_address,permanently_closed,\
                           name,opening_hours,geometry,types,place_id'}
    
    print('requesting query {}'.format(full_query))
    r = requests.get(URL, params=payload)
    req_count += 1
    textSearchResults = r.json()
    
    #fallout condition for when address doesn't work:
    if textSearchResults["status"] != "OK":
        
        #Create a broader query:
        fuzzy_query_list = [name, city, state, zipCd]
        fuzzy_query_list = [str(i) for i in fuzzy_query_list if not isinstance(i,type(None)) and str(i) !='nan']
        full_fuzzy_query = ' '.join(fuzzy_query_list)
        
        #check if we have enough data to proceed:
        if len(fuzzy_query_list) != 0:
        
            #retry request to textsearch:
            payload = {'key' : API_KEY,
                       'input' : full_fuzzy_query,
                       'inputtype' : 'textquery',
                       'fields' : 'photos,formatted_address,permanently_closed,\
                                   name,opening_hours,geometry,types,place_id'}

            print(' -- Status: {}. Re-trying query {}'.format(textSearchResults["status"], full_fuzzy_query))
            r = requests.get(URL, params=payload)
            req_count += 1
            textSearchResults = r.json()
            
            
        #If it still fails, return a not-found:
        if textSearchResults["status"] != "OK":
        
            if req_count == 2: 
                print('  -- Status: {} Hmm. Still not working. Skipping for now'.format(textSearchResults["status"]))
                status = 'RE-TRIED TWICE, NO RESULTS'
            elif req_count == 1:
                status = 'First try failed, not enough data to do a broader search'
                
            d = {'google_name' : ['NOT FOUND'],
                 'google_formatted_address' : ['NOT FOUND'],
                 'google_establishment_types' : ['NOT FOUND'],
                 'google_permanently_closed': ['NOT FOUND'],
                 'google_photo_reference_id' : ['NOT FOUND'],
                 'google_place_id' : ['NOT FOUND'],
                 'google_api_log': [status]}
            
            return req_count, d
        
        else:
            status = 'Found after re-trying second query: {}'.format(full_fuzzy_query)
    else:
        status = 'Found on first query using: {}'.format(full_query)
        
    #API Response:
    formatted_address = textSearchResults['candidates'][0]['formatted_address']
    types = [textSearchResults['candidates'][0]['types']]
    name = textSearchResults['candidates'][0]['name']
    closed = 'permanently_closed' in textSearchResults['candidates'][0].keys()

    if 'photos' in textSearchResults['candidates'][0].keys():
        photo_ref = textSearchResults['candidates'][0]['photos'][0]['photo_reference']
    else: photo_ref = 'No Photo'

    goog_place_id = textSearchResults['candidates'][0]['place_id']
        
    #dictionary to return
    d = {'google_name' : name,
         'google_formatted_address' : formatted_address,
         'google_establishment_types' : types,
         'google_permanently_closed': closed,
         'google_photo_reference_id' : photo_ref,
         'google_place_id' : goog_place_id,
         'google_api_log' : status}
    
    print(' -- found {}'.format(name + ' | '+formatted_address))
    return req_count, d


# Data Feeder Function

In [9]:
def feed_addresses(df, dataset='csg',start=0, stop=99):
    '''
    Function for feeding the addresses and customer names to the Places API from a specified dataset.
    
    Parameters
    ----------
    df (pandas dataframe): a dataframe object containing the addresses and customer names 
    dataset (string) : Label indicating which dataset. Allowed values are 'csg' or 'icoms'
    start (int) : specifies the row in the df at which to start. Default is 0
    start (stop) : specifies the row in the df at which to start Default is 99
    
    Returns
    -------
    pandas dataframe object : containing the key API attributes (columns) and name/address 
                              records fed to the API (rows)
    '''
    
    #Create new empty dataframe to capture the new API response:
    new_df = pd.DataFrame(columns=['google_name', 'google_formatted_address','google_establishment_types',
                                   'google_permanently_closed','google_photo_reference_id','google_place_id', 
                                   'google_api_log'])
    
    #Start the count for number of times we're hitting the API
    total_req = 0
    
    #Loop through the dataframe:
    for i, row in tqdm(df.loc[start:stop,:].iterrows(),total=stop-start,unit='records'): 
        print(i)
    
        try:
        #If CSG data is specified:
            if dataset.lower() == 'csg':
                req_count, api_response = single_query(row['ADDRESS_LINE_1'],
                                                       row['ADDRESS_LINE_2'],
                                                       row['CITY'],
                                                       row['STATE'],
                                                       row['ZIP_CODE'], 
                                                       name=row['CUSTOMER_NAME'])

        #If ICOMS data is specified: 
            elif dataset.lower() == 'icoms':

                req_count, api_response = single_query(row['STREET_ADDRESS'],
                                                       row['BUILDING'],
                                                       row['CITY'],
                                                       row['STATE'],
                                                       row['ZIP_CODE'], 
                                                       name = row['CUSTOMER_NAME'])
            else: 
                print('dataset name not valid') 
                break
        
        ## Error handling or Interrupting if needed ----------------------------------------------
        ## In the event of ANY type of error, or if the loop needs to stopped with a keyboard interrupt
        ## it will print the error message and exit safely while returning
        ## the dataframe that's been collected up until this point:
        except (Exception, KeyboardInterrupt) as e:
            msg = 'An error occured when processing line {}.\n -- Error type: {}.\n -- Error message: {} \
                  \n...Exiting loop and returning the dataframe up until that point.'
            print(msg.format(i, type(e).__name__, e.args))
            new_df = new_df.set_index(pd.Series(range(start,i)))
            return new_df
        
        #Add response data to the bottom of the collection
        new_df =  pd.concat([new_df, pd.DataFrame(api_response)],axis=0,ignore_index=True)
        total_req += req_count
    
    print('\nTotal # of API calls made: {}'.format(total_req))
    new_df = new_df.set_index(pd.Series(range(start,stop+1)))    
    return new_df
    

# Feed Addresses from CSG Data

In [7]:
api_df_csg = feed_addresses(df_csg, dataset='csg',start=678, stop=690)

678
requesting query ROSCOE POLICE DEPT 10595 MAIN ST   ROSCOE IL 61073-8830
 -- found Roscoe Police Department | 10595 Main St, Roscoe, IL 61073, United States
679
requesting query KINNIKINNICK SCHOOL 5410 PINE LN   ROSCOE IL 61073-7313
 -- found Kinnikinnick School | 5410 Pine Ln, Roscoe, IL 61073, United States
680
requesting query ROSCOE VILLAGE HALL 10631 MAIN ST   ROSCOE IL 61073-8564
 -- found Roscoe Village Clerk | 10631 Main St, Roscoe, IL 61073, United States
681
requesting query HARLEMROSCOE FIRE DEPT 13974 WILLOWBROOK RD   ROSCOE IL 61073-0000
 -- found Harlem Roscoe Fire Department Station 3 | 13974 Willowbrook Rd, Roscoe, IL 61073, United States
682
requesting query LEDGEWOOD ELEMENTARY SCH 11685 S GATE RD   ROSCOE IL 61073-9646
 -- found Ledgewood Elementary School | 11685 Southgate Rd, Roscoe, IL 61073, United States
683
requesting query STONE C SCHOOL 11633 S GATE RD   ROSCOE IL 61073-9646
 -- found Stone Creek Elementary School | 11633 Southgate Rd, Roscoe, IL 61073, 

# Feed Addresses from ICOMS Data

In [8]:
api_df_icoms = feed_addresses(df_icoms, dataset='icoms', start=3568, stop=3588)

3568
requesting query PS  DR CHARLES R DREW SCIENCE 50 A ST BUFFALO NY 14211
 -- found PS 59 Dr. Charles R. Drew Science Magnet Annex | 50 A St, Buffalo, NY 14211, United States
3569
requesting query MEDAILLE COLLEGE 18 AGASSIZ CIR BUFFALO NY 14214
 -- found Medaille College | 18 Agassiz Cir, Buffalo, NY 14214, United States
3570
requesting query NICHOLS SCHOOL 1250 AMHERST ST BUFFALO NY 14216
 -- found Nichols School | 1250 Amherst St, Buffalo, NY 14216, United States
3571
requesting query BOYS  GIRLS CLUB WC BAIRD 2061 BAILEY AVE BUFFALO NY 14211
 -- found Boys & Girls Clubs-Erie County | 2061 Bailey Ave, Buffalo, NY 14211, United States
3572
requesting query KENSINGTONBAILEY COMMUNITY CTR 2777 BAILEY AVE BUFFALO NY 14215
 -- found Edward Saunder Community Center | 2777 Bailey Ave, Buffalo, NY 14215, United States
3573
requesting query HOLY SPIRIT ACADEMY 85 DAKOTA ST BUFFALO NY 14216
 -- found Holy Spirit Church | 85 Dakota St, Buffalo, NY 14216, United States
3574
requesting query 

# Join API data back to the original dataset

In [114]:
df_csg_sample = pd.concat([df_csg.loc[api_df_csg.index[0]:api_df_csg.index[-1]],api_df_csg],axis=1)
df_icoms_sample = pd.concat([df_icoms.loc[api_df_icoms.index[0]:api_df_icoms.index[-1]],api_df_icoms],axis=1)

# Print to CSV

In [105]:
df_icoms_sample.to_csv('data/icoms_sample_PlacesAPI.csv',index=False)
df_csg_sample.to_csv('data/csg_sample_PlacesAPI.csv',index=False)